In [2]:
!pip install sqlalchemy==1.3.9
!pip install ibm_db_sa
!pip install ipython-sql
import sqlite3
import pandas as pd
from sqlite3 import Error
%load_ext sql
%sql sqlite:///sqlite-db-012022.db

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## 1

**Answer.**

In [9]:
def extract(query):
    con = %sql {query}
    df = con.DataFrame()
    return df
query = 'select salesorderdetail.orderqty, product.name  from salesorderdetail join product on salesorderdetail.productid = product.productid'
df = extract(query)
table = df.groupby('name', as_index=False).sum('orderqty')
table['promedio'] = table['orderqty']/266
table = table.sort_values(by='promedio', ascending=False).head(5)
table 


 * sqlite:///sqlite-db-012022.db
Done.


,name,orderqty,promedio
0,AWC Logo Cap,8311,31.244361
259,Water Bottle - 30 oz.,6815,25.620301
232,"Sport-100 Helmet, Blue",6743,25.349624
103,"Long-Sleeve Logo Jersey, L",6592,24.781955
231,"Sport-100 Helmet, Black",6532,24.556391


In [8]:
%sql SELECT salesorderdetail.orderqty, product.name  from salesorderdetail join product on salesorderdetail.productid = product.productid limit 5

 * sqlite:///sqlite-db-012022.db
Done.


orderqty,name
1,"Mountain-100 Black, 42"
3,"Mountain-100 Black, 44"
1,"Mountain-100 Black, 48"
1,"Mountain-100 Silver, 38"
1,"Mountain-100 Silver, 42"


**Answer.**

In [10]:
query = 'select businessentityid, salesytd from salesperson'
df = extract(query)
table = df.sort_values(by='salesytd', ascending=False).head(5)
table['businessentityid'] 

 * sqlite:///sqlite-db-012022.db
Done.


2     276
15    289
1     275
3     277
16    290
Name: businessentityid, dtype: int64

In [ ]:
'''En base a la tabla generada anteriormente se observa a los mejores vendedores con respecto a sus ventas realizadas hasta la fecha sin tener en cuenta el tipo de moneda'''

**Answer.**

In [11]:
query = 'select salespersonid, orderdate, subtotal from salesorderheader where orderdate between "2014-01-01" and "2014-12-31"'
df = extract(query)
table = df
df = df.groupby('salespersonid', as_index=False).sum()
table = df.sort_values(by='subtotal', ascending=False)
table['salespersonid'].head(5)

 * sqlite:///sqlite-db-012022.db
Done.


/tmp/ipykernel_12023/1381182898.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby('salespersonid', as_index=False).sum()


15    289.0
2     276.0
1     275.0
8     282.0
3     277.0
Name: salespersonid, dtype: float64

**Answer.**

-------

In [12]:
query = 'select salesorderheader.salespersonid, salesorderdetail.orderqty, salesorderdetail.unitprice from salesorderdetail join salesorderheader on salesorderdetail.salesorderid = salesorderheader.salesorderid where salesorderheader.orderdate between "2014-01-01" and "2014-12-31"'
df = extract(query)
table = df.groupby(['salespersonid', 'unitprice'], as_index=False).sum()
table['subtotal'] = table['orderqty'] * table['unitprice']
table = table.groupby('salespersonid', as_index=False).sum()
table = table.sort_values(by='subtotal', ascending=False)
table['salespersonid'].head(5)

 * sqlite:///sqlite-db-012022.db
Done.


15    289.0
2     276.0
1     275.0
8     282.0
3     277.0
Name: salespersonid, dtype: float64

### 5:

Use the tables: `countryregioncurrency`, `salesperson` and `salesterritory` to get the corresponding currency asociated with the top salespeople

**Answer.**

-------

In [13]:
query = 'select  salesperson.businessentityid, salesperson.salesytd,countryregioncurrency.currencycode from salesperson join salesterritory on salesperson.territoryid = salesterritory.territoryid join countryregioncurrency on countryregioncurrency.countryregioncode = salesterritory.countryregioncode'
df = extract(query)
table_ej_5 = df.sort_values(by='salesytd', ascending=False)
table_ej_5.head(5)

 * sqlite:///sqlite-db-012022.db
Done.


,businessentityid,salesytd,currencycode
1,276,4.251369e+06,USD
13,289,4.116871e+06,GBP
0,275,3.763178e+06,USD
2,277,3.189418e+06,USD
14,290,3.121616e+06,EUR


**Answer.**

-------

In [14]:
query = 'select averagerate, tocurrencycode from currencyrate where currencyratedate = "2011-11-30 00:00:00"'
df = extract(query)
df.rename(columns={'tocurrencycode': 'currencycode'}, inplace=True)
table = pd.merge(table_ej_5, df, how= 'left', on='currencycode')
table['salesytd'] = table['salesytd'] * table['averagerate']
table = table.sort_values(by='salesytd', ascending=False)
table['businessentityid'].head(5)

 * sqlite:///sqlite-db-012022.db
Done.


5    290
0    276
9    288
6    282
2    275
Name: businessentityid, dtype: int64